# Extending the fastai tabular regression model to handle missing and categorical data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sklearn.datasets as sk_data
import sklearn.model_selection as sk_selection
import sklearn.metrics as sk_metrics
import random_neural_net_models.tabular as rnnm_tab
import torch.optim as optim
import random_neural_net_models.losses as rnnm_loss
import random_neural_net_models.learner as rnnm_learner
import random_neural_net_models.data as rnnm_data
import random_neural_net_models.utils as rnnm_utils
from pathlib import Path
import torch
from torch.utils.data import DataLoader
from torch.utils.data import RandomSampler
import numpy as np

In [ ]:
rnnm_utils.make_deterministic(42)

In [ ]:
device = rnnm_utils.get_device()
device

## Generating data

In [ ]:
n_samples = 1_000
n_features = 3
n_informative = 2
n_targets = 1
X, y = sk_data.make_regression(
    n_samples=n_samples,
    n_features=n_features,
    random_state=42,
    n_informative=n_informative,
    n_targets=n_targets,
)

In [ ]:
X[:3], y[:3]

intentionally leaking the target class, if embedding is used properly model should pick up on it

In [ ]:
qs = np.linspace(10, 100, 10)
qs

In [ ]:
bins = np.percentile(y, q=qs)
y = np.digitize(y, bins)
X_cat = y.reshape((-1, 1))
y = y.astype(float)
X_cat[:3], y[:3]

In [ ]:
X_cat.shape, X.shape

In [ ]:
X_noise = np.random.normal(size=X.shape)
cols_with_missingness_num = (1,)
X_noise_miss, _ = rnnm_tab.make_missing_numerical(
    X_noise, p_missing=0.1, cols_with_missing=cols_with_missingness_num
)
X_noise_miss[:5, :]

In [ ]:
X_cat_miss, _ = rnnm_tab.make_missing_categorical(X_cat, p_missing=0.1)
X_cat_miss[:10, :]

In [ ]:
X0_num, X1_num, X0_cat, X1_cat, y0, y1 = sk_selection.train_test_split(
    X_noise_miss, X_cat_miss, y, test_size=0.2
)

In [ ]:
ds_train = rnnm_data.NumpyNumCatTrainingDataset(X0_num, X0_cat, y0)
ds_valid = rnnm_data.NumpyNumCatTrainingDataset(X1_num, X1_cat, y1)

In [ ]:
ds_train.cat_maps

In [ ]:
sampler = RandomSampler(
    ds_train,
    replacement=True,
    num_samples=int(1e5),
    generator=torch.manual_seed(3407),
)

In [ ]:
batch_size = 50
dl_train = DataLoader(
    ds_train,
    batch_size=batch_size,
    collate_fn=rnnm_data.collate_numpy_numcat_dataset_to_xyblock,
    sampler=sampler,
)
dl_valid = DataLoader(
    ds_valid,
    batch_size=batch_size,
    collate_fn=rnnm_data.collate_numpy_numcat_dataset_to_xyblock,
)

In [ ]:
next(iter(dl_train))

In [ ]:
next(iter(dl_train)).x_categorical[:10]

## TabularModelRegression

In [ ]:
n_categories_per_column = rnnm_data.calc_n_categories_per_column(X0_cat)
n_categories_per_column

In [ ]:
y0.min(), y0.max()

In [ ]:
n_hidden = [200, 100]

do_impute = True
impute_bias_source = rnnm_tab.BiasSources.zero
n_features_in = n_features + 1
mean = y0.mean()
std = y0.std()
model = rnnm_tab.TabularModelRegression(
    n_features=n_features_in,
    n_hidden=n_hidden,
    mean=mean,
    std=std,
    use_batch_norm=False,
    do_impute=do_impute,
    n_categories_per_column=n_categories_per_column,
    impute_bias_source=impute_bias_source,
    cols_with_missing=cols_with_missingness_num,
)

In [ ]:
learning_rate = 0.1
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss = rnnm_loss.MSELossXy()
loss_callback = rnnm_learner.TrainLossCallback()

save_dir = Path("./models")

callbacks = [loss_callback]

In [ ]:
learner = rnnm_learner.Learner(
    model,
    optimizer,
    loss,
    callbacks=callbacks,
    save_dir=save_dir,
    device=device,
    show_epoch_progress=True,
)

In [ ]:
lr_find_callback = rnnm_learner.LRFinderCallback(1e-5, 100, 100)

learner.find_learning_rate(
    dl_train, n_epochs=10, lr_find_callback=lr_find_callback
)

In [ ]:
lr_find_callback.plot(yscale="log")

In [ ]:
learning_rate = 1e-3
n_epochs = 2

scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer=optimizer,
    max_lr=learning_rate,
    epochs=n_epochs,
    steps_per_epoch=len(dl_train),
)
scheduler_callback = rnnm_learner.EveryBatchSchedulerCallback(scheduler)
learner.update_callback(scheduler_callback)

In [ ]:
learner.fit(dl_train, n_epochs=n_epochs, dataloader_valid=dl_valid)

In [ ]:
loss_callback.plot()

In [ ]:
preds = learner.predict(dl_valid).detach().numpy()
preds[:3]

In [ ]:
y1[:3]

In [ ]:
print(sk_metrics.mean_squared_error(y1, preds))